In [1]:
import os
os.chdir('../')

import transtab

# set random seed
transtab.random_seed(42)

/Users/lushinchen/Documents/projects/tabular/transfer_learning/transtab_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# load a dataset and start vanilla supervised training
allset, trainset, valset, testset, cat_cols, num_cols, bin_cols = transtab.load_data(['credit-g', 'credit-approval'])

# build transtab classifier model
model = transtab.build_classifier(cat_cols, num_cols, bin_cols, device='cpu')

# start training
training_arguments = {
    'num_epoch':50,
    'eval_metric':'val_loss',
    'eval_less_is_better':True,
    'output_dir':'./checkpoint',
    'batch_size':128,
    'lr':1e-4,
    'weight_decay':1e-4,
    }
transtab.train(model, trainset[0], valset[0], **training_arguments)

# save model
model.save('./ckpt/pretrained')

########################################


/Users/lushinchen/Documents/projects/tabular/transfer_learning/transtab/transtab/dataset.py:159: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  dataset = openml.datasets.get_dataset(dataname)


openml data index: 31
load data from credit-g
# data: 1000, # feat: 20, # cate: 11,  # bin: 2, # numerical: 7, pos rate: 0.70
########################################


/Users/lushinchen/Documents/projects/tabular/transfer_learning/transtab/transtab/dataset.py:192: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  for col in num_cols: X[col].fillna(X[col].mode()[0], inplace=True)
/Users/lushinchen/Documents/projects/tabular/transfer_learning/transtab/transtab/dataset.py:192: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the interme

openml data index: 29
load data from credit-approval
# data: 690, # feat: 15, # cate: 9,  # bin: 0, # numerical: 6, pos rate: 0.56


...

epoch: 30, test val_loss: 0.462955
EarlyStopping counter: 3 out of 5
epoch: 30, train loss: 2.6546, lr: 0.000100, spent: 52.7 secs


Epoch:  64%|██████▍   | 32/50 [00:54<00:30,  1.69s/it]

epoch: 31, test val_loss: 0.437068
EarlyStopping counter: 4 out of 5
epoch: 31, train loss: 2.6599, lr: 0.000100, spent: 54.3 secs


Epoch:  64%|██████▍   | 32/50 [00:55<00:31,  1.75s/it]
2024-03-08 15:41:21.629 | INFO     | transtab.trainer:train:136 - load best at last from ./checkpoint
2024-03-08 15:41:21.643 | INFO     | transtab.trainer:save_model:247 - saving model checkpoint to ./checkpoint
2024-03-08 15:41:21.787 | INFO     | transtab.trainer:train:141 - training complete, cost 56.1 secs.


epoch: 32, test val_loss: 0.486147
EarlyStopping counter: 5 out of 5
early stopped


In [4]:
# now let's use another data and try to leverage the pretrained model for finetuning
# here we have loaded the required data `credit-approval` before, no need to load again.

# load the pretrained model
model.load('./ckpt/pretrained')

# update model's categorical/numerical/binary column dict
# need to specify the number of classes if the new dataset has different # of classes from the 
# pretrained one.
model.update({'cat':cat_cols,'num':num_cols,'bin':bin_cols, 'num_class':2})

2024-03-08 15:41:31.791 | INFO     | transtab.modeling_transtab:load:774 - missing keys: []
2024-03-08 15:41:31.792 | INFO     | transtab.modeling_transtab:load:775 - unexpected keys: []
2024-03-08 15:41:31.793 | INFO     | transtab.modeling_transtab:load:776 - load model from ./ckpt/pretrained
2024-03-08 15:41:31.815 | INFO     | transtab.modeling_transtab:load:222 - load feature extractor from ./ckpt/pretrained/extractor/extractor.json


In [6]:
# start training
training_arguments = {
    'num_epoch':50,
    'eval_metric':'auc',
    'eval_less_is_better':False,
    'output_dir':'./checkpoint',
    'batch_size':128,
    'lr':2e-4,
    }

transtab.train(model, trainset[1], valset[1], **training_arguments)


Epoch:   2%|▏         | 1/50 [00:00<00:36,  1.35it/s]

epoch: 0, test auc: 0.918615
epoch: 0, train loss: 1.7609, lr: 0.000200, spent: 0.7 secs


Epoch:   4%|▍         | 2/50 [00:01<00:28,  1.67it/s]

epoch: 1, test auc: 0.914286
EarlyStopping counter: 1 out of 5
epoch: 1, train loss: 1.7883, lr: 0.000200, spent: 1.2 secs


Epoch:   6%|▌         | 3/50 [00:01<00:29,  1.60it/s]

...

epoch: 7, test auc: 0.912554
EarlyStopping counter: 3 out of 5
epoch: 7, train loss: 1.5259, lr: 0.000200, spent: 4.8 secs


Epoch:  18%|█▊        | 9/50 [00:05<00:22,  1.82it/s]

epoch: 8, test auc: 0.913420
EarlyStopping counter: 4 out of 5
epoch: 8, train loss: 1.5106, lr: 0.000200, spent: 5.3 secs


Epoch:  18%|█▊        | 9/50 [00:05<00:26,  1.55it/s]
2024-03-08 15:42:21.218 | INFO     | transtab.trainer:train:136 - load best at last from ./checkpoint
2024-03-08 15:42:21.223 | INFO     | transtab.trainer:save_model:247 - saving model checkpoint to ./checkpoint
2024-03-08 15:42:21.307 | INFO     | transtab.trainer:train:141 - training complete, cost 5.9 secs.


epoch: 9, test auc: 0.910823
EarlyStopping counter: 5 out of 5
early stopped


In [9]:
# evaluation
x_test, y_test = testset[1]
ypred = transtab.predict(model, x_test, y_test)
transtab.evaluate(ypred, y_test, metric='auc')

from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_test, ypred))

0.8639557164147328
